<a href="https://colab.research.google.com/github/BOD-27/GP/blob/main/final_face_recognentionGPLAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install MTCNN library**

**Install keras-facenet**


In [1]:
!pip install mtcnn
!pip install keras-facenet

**Required libraries**

In [2]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from scipy.spatial.distance import cosine

**Suppress TensorFlow warnings**

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

**FaceLoading class for loading and processing images**

In [4]:
class FaceLoading:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.x = []
        self.y = []
        self.detector = MTCNN()

    def ExtractFace(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x, y, w, h = self.detector.detect_faces(img)[0]['box']
        x, y = abs(x), abs(y)
        face = img[y:y + h, x:x + w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr




    def LoadFace(self, file_path):
        faces = []
        try:
            single_face = self.ExtractFace(file_path)
            faces.append(single_face)
        except Exception as e:
            pass
        return faces


    def LoadClasses(self):
        for file_name in os.listdir(self.directory):
            if file_name.endswith('.jpg') or file_name.endswith('.png') or file_name.endswith('.jpeg'):  # Adjust file extensions as per your dataset
                file_path = os.path.join(self.directory, file_name)
                faces = self.LoadFace(file_path)  # Pass the file path directly
                labels = [file_name] * len(faces)
                print(f"Loaded successfully: {len(labels)}")
                self.x.extend(faces)
                self.y.extend(labels)

        return np.asarray(self.x), np.asarray(self.y)




 **Load and process images for face** **recognition**

 **Load pre-trained FaceNet model**

In [5]:
embedder = FaceNet()

def GetEmbedding(face_img):
    face_img = face_img.astype('float32') #3D (160x160x3)
    face_img = np.expand_dims(face_img, axis=0)
    #4D (nonex160x160x3)
    yhat = embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

**Define a function to find closest matches**

In [7]:
def find_closest_matches(embedded_input, embedded_dataset, dataset_labels, top_n=7):
    distances = []
    outlabels = []

    for input_embedding in embedded_input:
        # Compute distances between input embedding and embeddings in the dataset
        distances_for_input = [cosine(input_embedding, dataset_embedding) for dataset_embedding in embedded_dataset]

        # Find the index with minimum distance
        min_distance_index = np.argmin(distances_for_input)

        # Append the distance and label of the closest match
        distances.append(distances_for_input[min_distance_index])
        outlabels.append(dataset_labels[min_distance_index])

    # Sort distances and corresponding labels in ascending order
    sorted_indices = np.argsort(distances)
    sorted_distances = np.array(distances)[sorted_indices]
    sorted_labels = np.array(outlabels)[sorted_indices]

    # Get top n closest matches
    top_matches = [(sorted_labels[i], sorted_distances[i]) for i in range(top_n)]

    return top_matches



In [12]:
from collections import Counter
import re

def preprocess_label(label):
    # Remove numbers
    label = re.sub(r'\d+', '', label)
    # Remove periods and file extensions (e.g., .jpg, .png, .jpeg, etc.)
    label = re.sub(r'\.\w{3,4}|\.', '', label)
    # Remove brackets and anything inside them
    label = re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', label)
    # Remove general suffixes after an underscore
    label = re.sub(r'_[\w\d]+.*$', '', label)
    # Remove leading and trailing whitespace
    label = label.strip()
    return label
# Assuming top_matches contains the top matches
def check_for_match(top_matches):
    # Extract labels and distances
    labels = [preprocess_label(label) for label, _ in top_matches]
    distances = [distance for _, distance in top_matches]
    print(labels)

    # get most frequent labels
    frequent_labels = Counter(labels).most_common()
    print (frequent_labels)

    # Check if any frequent label occurs 4 or more times and average cosine similarity is less than 0.5
    for label, count in frequent_labels:
        if count >= 4:
            label_distances = [distance for orig_label, distance in top_matches if preprocess_label(orig_label) == label]
            avg_similarity = sum(label_distances) / count
            if avg_similarity < 0.5:
                return True, avg_similarity, label

        elif count == 3:
            # Check if the 3rd most frequent label is the first one
           # if preprocess_label(top_matches[0][0]) == label:
                label_distances = [distance for orig_label, distance in top_matches if preprocess_label(orig_label) == label]
                avg_similarity = sum(label_distances) / count
                if avg_similarity < 0.5:
                  return True, avg_similarity, label



        else :
          return False, None, None



In [13]:
def main():


    # Load and process images for face recognition
    faceLoading = FaceLoading('/content/Final_dataset')
    X, y = faceLoading.LoadClasses()

    # Save labels for future reference
    labels = y



    # Extract embeddings for the loaded images
    embedded_x = [GetEmbedding(img) for img in X]
    embedded_x = np.asarray(embedded_x)



    # Load and process new images for comparison
    faceLoading2 = FaceLoading('/content/FinalTest_dataset/JodyKhaled')
    X1, y1 = faceLoading2.LoadClasses()

    # Display labels
    labels1 = y1



    # Compute embeddings for new input images
    embedded_x1 = [GetEmbedding(img) for img in X1]
    embedded_x1 = np.asarray(embedded_x1)

    # Find closest matches
    top_matches = find_closest_matches(embedded_x1, embedded_x, labels, top_n=7)

    # Print top 7 similar images and their similarity scores
    for i, (label, distance) in enumerate(top_matches, 1):
        print(f"Similar Image {i}: Label - {label}, Similarity Score - {distance}")

    # Check for match
    match_found, avg_similarity, matched_label = check_for_match(top_matches)

    if match_found:
        print("Match found with label:", matched_label)
        print("Average cosine similarity:", avg_similarity)
    else:
        print("No match found.")

if __name__ == "__main__":
    main()

1/1 [==============================] - 0s 119ms/step


1/1 [==============================] - 0s 154ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 22ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 21ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 20ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 21ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 21ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 33ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 36ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 33ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 20ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 23ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 21ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 24ms/step
Loaded successfully: 1
1/1 [===================